In [3]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import numpy as np

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from config import config

In [4]:
config=config()
engine=create_engine("postgresql://"+config["user"]+":"+config["password"]+"@"+config["host"]+"/"+config["database"])

In [7]:
sql=("SELECT lt.daystamp, lt.trip_id, lt.stoppoint_id,lt.departure_time_p,"
    "lt.departure_time_a,trips.departure_time_p,trips.departure_time_a,"
     "line_id,route_id,weather_main,temp "
     "FROM leavetimes AS lt, trips, weather "
     "WHERE trips.line_id='46A' AND trips.direction=1 AND trips.suppressed=0 "
     "AND lt.daystamp = trips.daystamp AND lt.trip_id = trips.trip_id AND lt.suppressed=0"
     "AND lt.weather_id = weather.daytime")

In [8]:
df = pd.read_sql(sql,engine)

In [9]:
df.shape

(2145060, 11)

In [10]:
df.head()

,daystamp,trip_id,stoppoint_id,departure_time_p,departure_time_a,departure_time_p,departure_time_a,line_id,route_id,weather_main,temp
0,1517961600,6253924,807,60960,60926,60960,60926.0,46A,74,Rain,3.08
1,1517961600,6253924,808,61053,61036,60960,60926.0,46A,74,Rain,3.08
2,1517961600,6253924,809,61107,61114,60960,60926.0,46A,74,Rain,3.08
3,1517961600,6253924,810,61193,61215,60960,60926.0,46A,74,Rain,3.08
4,1517961600,6253924,811,61246,61254,60960,60926.0,46A,74,Rain,3.08


In [11]:
df.columns=["daystamp","trip_id","stop_id","dep_p","dep_a","start_p","start_a","line_id","route_id","weather","temp"]

In [12]:
df.head()

,daystamp,trip_id,stop_id,dep_p,dep_a,start_p,start_a,line_id,route_id,weather,temp
0,1517961600,6253924,807,60960,60926,60960,60926.0,46A,74,Rain,3.08
1,1517961600,6253924,808,61053,61036,60960,60926.0,46A,74,Rain,3.08
2,1517961600,6253924,809,61107,61114,60960,60926.0,46A,74,Rain,3.08
3,1517961600,6253924,810,61193,61215,60960,60926.0,46A,74,Rain,3.08
4,1517961600,6253924,811,61246,61254,60960,60926.0,46A,74,Rain,3.08


In [13]:
def daystamp_converter(time):
    date=datetime.fromtimestamp(time)
    return (date.weekday(),date.month,date.hour)

In [14]:
df["datetime"]=df.daystamp.values+df.dep_p.values

In [15]:
df["weekday"],df["month"],df["hour"]=zip(*df['datetime'].apply(daystamp_converter))

In [16]:
df.head()

,daystamp,trip_id,stop_id,dep_p,dep_a,start_p,start_a,line_id,route_id,weather,temp,datetime,weekday,month,hour
0,1517961600,6253924,807,60960,60926,60960,60926.0,46A,74,Rain,3.08,1518022560,2,2,16
1,1517961600,6253924,808,61053,61036,60960,60926.0,46A,74,Rain,3.08,1518022653,2,2,16
2,1517961600,6253924,809,61107,61114,60960,60926.0,46A,74,Rain,3.08,1518022707,2,2,16
3,1517961600,6253924,810,61193,61215,60960,60926.0,46A,74,Rain,3.08,1518022793,2,2,16
4,1517961600,6253924,811,61246,61254,60960,60926.0,46A,74,Rain,3.08,1518022846,2,2,17


In [17]:
df["dur_s"]=df.dep_p.values-df.start_p.values
df["dur_a"]=df.dep_a.values-df.start_a.values

In [18]:
df_ml=df.drop(["daystamp","trip_id","dep_p","dep_a","start_p","start_a","datetime"],axis=1)

Filter out unusual routes:

In [19]:
route_counts=df_ml.route_id.value_counts()
indices=route_counts.index
values=route_counts.values
cum_value=0
size=df_ml.shape[0]
index=1
for value in values:
    cum_value+=value
    ratio=cum_value/size
    print(f"{index} most common route(s) cover {ratio*100:.2f}% of the routes.")
    if(ratio>0.8):
        break
    index+=1

1 most common route(s) cover 97.63% of the routes.


In [20]:
routes=indices[:index]
routes

Int64Index([74], dtype='int64')

In [21]:
df_ml=df_ml[df_ml.route_id.isin(routes)]

In [22]:
df_ml.shape

(2094285, 10)

Check for null values

In [23]:
df_ml.isnull().sum()

stop_id         0
line_id         0
route_id        0
weather         0
temp            0
weekday         0
month           0
hour            0
dur_s           0
dur_a       64155
dtype: int64

In [24]:
df_ml.isnull().sum().sum()/df_ml.shape[0]

0.030633366518883532

In [25]:
df_ml = df_ml.dropna(axis = 0, how ='any') 

See how resulting stops line up:

In [26]:
stop_counts=df_ml.stop_id.value_counts()

In [27]:
stop_counts

807     35117
811     34929
814     34927
813     34924
81      34920
812     34920
810     34903
809     34887
818     34887
819     34881
817     34879
264     34805
334     34759
808     34732
406     34703
757     34688
756     34687
848     34685
758     34681
2795    34681
847     34677
759     34673
6059    34632
846     34621
842     34575
845     34529
747     34525
2007    34428
2009    34414
2008    34390
2010    34388
435     34335
7353    34321
4571    34216
2013    34170
2014    34143
4636    34136
2016    34134
760     34128
2015    34123
2017    34120
2018    34111
2022    34105
2021    34105
2019    34103
2020    34098
4565    34091
4566    34090
4567    34089
2032    34069
2031    34069
2034    34052
2033    34050
2035    34042
2039    33994
763     33908
762     33819
2036    33782
761     33280
Name: stop_id, dtype: int64

Finally, drop line_id and route_id (both have cardinality 1 at this point)

In [28]:
df_ml=df_ml.drop(["line_id","route_id"],axis=1)

In [29]:
df_ml.head()

,stop_id,weather,temp,weekday,month,hour,dur_s,dur_a
0,807,Rain,3.08,2,2,16,0,0.0
1,808,Rain,3.08,2,2,16,93,110.0
2,809,Rain,3.08,2,2,16,147,188.0
3,810,Rain,3.08,2,2,16,233,289.0
4,811,Rain,3.08,2,2,17,286,328.0


In [30]:
df_ml_810=df[df.stop_id==810]

In [31]:
df_ml_810

,daystamp,trip_id,stop_id,dep_p,dep_a,start_p,start_a,line_id,route_id,weather,temp,datetime,weekday,month,hour,dur_s,dur_a
3,1517961600,6253924,810,61193,61215,60960,60926.0,46A,74,Rain,3.08,1518022793,2,2,16,233,289.0
62,1517961600,6262164,810,24694,24930,24480,24697.0,46A,74,Clouds,-1.60,1517986294,2,2,6,214,233.0
121,1520726400,6397655,810,83817,83837,83700,83653.0,46A,74,Rain,6.99,1520810217,6,3,23,117,184.0
180,1520726400,6392156,810,77517,77506,77400,77400.0,46A,74,Rain,6.99,1520803917,6,3,21,117,106.0
239,1520726400,6392515,810,75717,75740,75600,75633.0,46A,74,Clouds,6.95,1520802117,6,3,21,117,107.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2144739,1526338800,6764906,810,45301,45313,45120,45157.0,46A,74,Clouds,16.27,1526384101,1,5,12,181,156.0
2144762,1526338800,6763274,810,35251,35283,35040,35040.0,46A,74,Clouds,15.13,1526374051,1,5,9,211,243.0
2144821,1526338800,6753808,810,57324,57232,57120,57111.0,46A,74,Clouds,14.78,1526396124,1,5,15,204,121.0
2144928,1526252400,6763192,810,60515,60581,60300,60273.0,46A,74,Clouds,14.70,1526312915,0,5,16,215,308.0


In [32]:
df_weekday=pd.get_dummies(df_ml.weekday)
df_month=pd.get_dummies(df_ml.month)
df_hour=pd.get_dummies(df_ml.hour)
df_weather=pd.get_dummies(df_ml.weather)
df_stops=pd.get_dummies(df_ml.stop_id)

In [33]:
#setting up splits for prediction of scheduled duration
y_prior=df_ml["dur_s"]
df_test_prior=pd.concat([df_weekday,df_month,df_hour,df_stops],axis=1)
X=df_test_prior
X_train,X_test,y_train,y_test=train_test_split(X,y_prior,random_state=1)

In [ ]:
regr_relu=MLPRegressor(activation="relu",random_state=1, max_iter=100, verbose=1).fit(X_train, y_train)

In [36]:
regr_relu.score(X_test, y_test)

0.9881124114587736

Above's neural network took about 10 mins to train. If schedule info isn't available elsewhere, we could use one predictor for the scheduled time and a second for the actual after the scheduled time was predicted.

In [34]:
reg = LinearRegression().fit(X_train, y_train)

In [35]:
reg.score(X_test, y_test)

0.963409408709

For comparison, a linear regressor takes about 5 secs to train and make predictions

In [36]:
y=df_ml["dur_a"]
df_test=df_ml.drop(["stop_id","weather","dur_a"],axis=1)

In [37]:
df_test.head()

,temp,weekday,month,hour,dur_s
0,3.08,2,2,16,0
1,3.08,2,2,16,93
2,3.08,2,2,16,147
3,3.08,2,2,16,233
4,3.08,2,2,17,286


In [38]:
df_test=df_test.drop(["weekday","month","hour"],axis=1)

In [39]:
df_test=pd.concat([df_test,df_weather,df_weekday,df_month,df_hour],axis=1)

In [40]:
df_test.shape

(2030130, 49)

In [41]:
X=df_test

In [42]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1)

In [50]:
regr_relu=MLPRegressor(activation="relu",random_state=1, max_iter=100, verbose=1).fit(X_train, y_train)

Iteration 1, loss = 119204.66950077
Iteration 2, loss = 71929.71894800
Iteration 3, loss = 70293.85671083
Iteration 4, loss = 69369.41826543
Iteration 5, loss = 68973.24066100
Iteration 6, loss = 68594.58577374
Iteration 7, loss = 68641.13589531
Iteration 8, loss = 68392.02902936
Iteration 9, loss = 68335.92583998
Iteration 10, loss = 68103.48492222
Iteration 11, loss = 68105.74877054
Iteration 12, loss = 67933.58056800
Iteration 13, loss = 67843.77519515
Iteration 14, loss = 67728.06374080
Iteration 15, loss = 67692.75523335
Iteration 16, loss = 67499.14183942
Iteration 17, loss = 67448.09497662
Iteration 18, loss = 67436.78215532
Iteration 19, loss = 67222.37520219
Iteration 20, loss = 67137.80627570
Iteration 21, loss = 67196.58358438
Iteration 22, loss = 67033.63211109
Iteration 23, loss = 66910.67694706
Iteration 24, loss = 66881.95982112
Iteration 25, loss = 66881.48469560
Iteration 26, loss = 66880.34302119
Iteration 27, loss = 66716.85660354
Iteration 28, loss = 66619.22348658


c:\python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


In [51]:
regr_relu.score(X_test, y_test)

0.9376675098304893

Model above took about 12 mins to train

First model try took almost 2 hours to run, good starting value, but below 64k after around 100 iterations and ended up at 61k, converging after 430 iterations. accuracy of 0.937145. That model included stop_id dummy encoded and day instead of workday. none of the dates were dummy encoded.

In [43]:
reg = LinearRegression().fit(X_train, y_train)

In [44]:
reg.score(X_test, y_test)

0.9325282724189506